# Labo 8 : modèles de langage / Cloze Test
Par Johanna Melly et Yohann Meyer

## Objectif et plan

L'objectif de ce labo est d'entraîner des modèles de langues (EN : language models) sous NLTK en utilisant le package nltk.lm. Les modèles seront entraînés sur des romans de Balzac en français
(fournis par le projet Gutenberg), et leurs performances seront mesurées par leur perplexité sur de nouveaux textes. Les modèles serviront également à deviner des mots cachés dans un texte, et ici leurs performances seront comparées à celles des humains sur cette même tâche.

## Importation

In [1]:
import antigravity

## Obtenir les données

Télécharger les dix romans de la Comédie humaine de Balzac en français, enlever les notices en anglais au début et à la fin, puis les assembler en un corpus.  
**NOTE : nous remercions le groupe de Vincent Guidoux et Nathan Gonzalez qui nous ont gentiment partagé leur fichier la_comedie_humaine.txt alors que le site de Gutenberg était down, afin que nous puissions tout de même réaliser le laboratoire**

In [6]:
from urllib import request

baseUrl = "http://www.gutenberg.org/files/"
comedieHumaine = ["43851/43851-0.txt", "45060/45060-0.txt", "48082/48082-0.txt", "49482/49482-0.txt", "51381/51381-0.txt", "52831/52831-0.txt", "54723/54723-0.txt", "55860/55860-0.txt", "58244/58244-0.txt"]
corpus = ''

for volume in comedieHumaine:
    response = request.urlopen(baseUrl+volume)
    raw = response.read().decode('utf-8')
    start = raw.find('Au lecteur')
    end = raw.rfind('End of')
    corpus+=raw[start:end]


KeyboardInterrupt: 

Sauvegarder le texte résultant : quelle taille fait-il en Ko ou Mo?

In [29]:
import os, codecs
corpus = open('data/la_comedie_humaine.txt', 'r', encoding='utf-8').read()
print("Taille du fichier: " + str(os.path.getsize('data/la_comedie_humaine.txt')/1000000) + ' MB')

Taille du fichier: 12.552112 MB


Segmenter et tokenizer chacun des deux corpus (train/test) en une liste de listes de mots, y compris les ponctuations (une liste = une phrase).

In [70]:
import nltk
sents = nltk.sent_tokenize(corpus, language='french')
wordSents = [nltk.word_tokenize(s, language='french') for s in sents]
sents

['\n\n\n\n\n\n\n\n\n\n  au lecteur\n\n  cette version numérisée reproduit, dans son intégralité,\n  la version originale.',
 "la ponctuation n'a pas été modifiée hormis quelques corrections\n  mineures.",
 "l'orthographe a été conservée.",
 'seuls quelques mots ont été modifiés.',
 'la liste des modifications se trouve à la fin du texte.',
 'oeuvres complètes\n  de\n  h. de balzac\n\n\n  la comédie humaine\n\n  premier volume\n\n\n  première partie\n  études de moeurs\n\n\n  premier livre\n\n\n  paris.--imprimerie de e. martinet, rue mignon, 2\n\n  [illustration: balzac]\n\n\n\n\n  scènes\n  de\n  la vie privée\n\n  tome 1\n\n\n  la maison du chat-qui-pelote--le bal de sceaux\n  la bourse--la vendetta--madame firmiani\n  une double famille--la paix du ménage--la fausse maitresse\n  étude de femme--albert savarus\n\n\n  paris\n\n  veuve andre houssiaux, éditeur\n\n  hébert et cie, successeurs\n  7, rue perronet, 7\n\n  1874\n\n\n\n\nhonoré de balzac\n\n\nbalzac naquit à tours le 16 mai 

In [44]:
nbToken = sum([len(x) for x in wordSents])
nbToken

2335258

Extraire un fragment (environ 2000 mots) qui servira de donnée de test (pas le bon format, utiliser C.H. 10).

In [93]:
import random
nbWordsTaken = 0
randomizedSet = wordSents
random.shuffle(randomizedSet)
trainSet = []
testSet = []

for sent in randomizedSet:
    nbWordsTaken += len(sent)
    if(nbWordsTaken < 2000):
        testSet.append(sent)
    else:
        index = randomizedSet.index(sent)
        break
trainSet = randomizedSet[index:]

print("Taille train set: " + str(len(trainSet)) + "\nTaille test set: " + str(len(testSet)))

Taille train set: 101143
Taille test set: 95


In [96]:
print("Nombre de mots dans le set de test: " + str(sum([len(x) for x in testSet])))

Nombre de mots dans le set de test: 1993


Sauvegarder aussi chaque liste dans un fichier pickle.

In [106]:
import pickle 
file_pickle1 = open('tarinSet.obj', 'wb')
file_pickle2 = open('testSet.obj', 'wb')
pickle.dump(trainSet, file_pickle1)
pickle.dump(testSet, file_pickle2)

## Entraîner un premier modèle de langage de NLTK

En utilisant les instructions disponibles pour le module NLTK LM, entraîner un modèle de langage sur les données d'entraînement. Attention, le package lm n'est disponible qu'à partir de NLTK version 3.4.

In [122]:
from nltk.lm import MLE
from nltk.lm import Vocabulary

Commencer avec un modèle MLE à l'ordre 2, comme montré dans le tutoriel.

In [130]:
# paddedLine found at https://stackoverflow.com/questions/54959340/nltk-language-modeling-confusion
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(2, trainSet)

In [131]:
lm = MLE(2)
lm.fit(train, vocab)
lm.generate(3)

[',', 'des', 'vertus']

Puis, par exemple, aller jusqu'à l'ordre 3 ou 4, avec l'algorithme de Laplace (ou de Kneser-Ney).

In [132]:
train, vocab = padded_everygram_pipeline(4, trainSet)
lm = MLE(4)
lm.fit(train, vocab)

Calculer la perplexité du modèle sur l’ensemble de test.

In [134]:
lm.perplexity(testSet)

inf

Notre résultat est infini car certains mots du test set ne font pas partie du vocabulaire défini.

Générer quelques phrases dans le style de Balzac selon les explications de NLTK.

In [136]:
lm.generate(text_seed=['Je', 'ne','suis', 'pas'], num_words=2)

['assez', 'forte']

In [145]:
lm.generate(text_seed=['Quand', 'il'], num_words=8)

['descendit', 'en', 'emportant', 'cette', 'pièce', 'de', 'conviction', ',']

In [149]:
print(lm.generate(text_seed=['Quand', 'il'], num_words=8))

["s'aperçut", "qu'il", 'avait', 'plu', 'à', 'tout', 'le', 'monde']


In [157]:
lm.generate(text_seed=['Rien', 'ne', 'pourra', 'jamais'], num_words=3)

['greffer', 'un', 'livre']

In [172]:
lm.generate(text_seed=['J\'aime' 'bien'], num_words=2)

['la', 'grâce']

In [181]:
print("Ma sœur n'est qu'une " + str(lm.generate(text_seed=['Ma', 'sœur', 'n\'est', 'qu\'une'], num_words=1)))

Ma sœur n'est qu'une scélérate


In [193]:
print("Yohann Meyer est très, très " + str(lm.generate(text_seed=['Yohann', 'Meyer', 'est', 'très', ',', 'très'], num_words=1)))

Yohann Meyer est très, très usé


## Entraîner un second modèle de langage de NLTK

En utilisant par exemple le modèle de Laplace.

In [198]:
from nltk.lm import Laplace
lm2 = Laplace(4)

Le tester aussi sur le corpus de test et comparer les scores.

In [205]:
lm2.fit(train, vocab)

In [207]:
lm2.unmasked_score

<bound method Lidstone.unmasked_score of <nltk.lm.models.Laplace object at 0x7fc276763240>>

## Cloze test

Supprimer un mot sur 7 dans le corpus de test.

En utilisant la méthode lm.generate avec du contexte, demander au modèle de langage de prédire ces mots (en utilisant donc 2-4 mots précédents). Quelle est la performance du système ? Comparer les modèles générés en (2) et en (3).

Générez un texte avec les mots en question (1 mot sur 7) remplacés par des blancs, et passez-le à un autre binôme : quelle est leur performance pour deviner les mots ?

Merci d’envoyer votre notebook Jupyter par email au professeur avant le vendredi 14 juin à 23h59.